In [51]:
import random, re, csv
import numpy as np
from sklearn import metrics,svm
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from simdata import *
from interdata_mysql_simple import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score

#get data from csv files
mixdata_sim_filedir='../resources/gamedata/mixdata_sim.csv'
inerdata_sim_filedir='../resources/gamedata/interdata_sim.csv'
biodata_sim_filedir='../resources/gamedata/biodata_sim.csv'


mixdata_sim=get_mldata(mixdata_sim_filedir)
interdata_sim=get_mldata(inerdata_sim_filedir)
biodata_sim=get_mldata(biodata_sim_filedir)

In [52]:
# make data for ml analysis
features_interdata_sim=[]
labels_interdata_sim_true=[]
features_biodata_sim=[]
labels_biodata_sim_true=[]

for i in interdata_sim:
    features_interdata_sim=features_interdata_sim+[i[0:4]]
    labels_interdata_sim_true=labels_interdata_sim_true+[i[4]]
for i in biodata_sim:
    features_biodata_sim=features_biodata_sim+[i[0:10]]
    labels_biodata_sim_true=labels_biodata_sim_true+[i[10]]

# cluster interdata by kmeans 
train_features_km=np.array(features_interdata_sim[0:400])
clu_km=KMeans(n_clusters=2,random_state=0).fit(train_features_km)
#labels_pre_km=kmeans.predict(features_interdata_sim[400:579])
#testlabels_true=label_interdata_sim[400:579]
#metrics.adjusted_rand_score(testlabels_true,testlabels_pre_km)

#labels for biodata
labels_biodata_sim_pre_km=clu_km.predict(features_interdata_sim)
#metrics.adjusted_rand_score(labels_interdata_sim_true,labels_biodata_sim_pre_km)
diff_num=[]
'''for i in range(len(labels_biodata_sim_pre_km)):
    diff=labels_biodata_sim_pre_km[i]-labels_interdata_sim_true[i]
    if diff!=0:
        diff_num=diff_num+[diff]'''
#print(diff_num)


#classify biodata by SVM
train_features_svm=np.array(features_biodata_sim[0:400])
train_labels_svm=np.array(labels_biodata_sim_pre_km[0:400])
test_features_svm=np.array(features_biodata_sim[400:580])
test_labels_svm=np.array(labels_biodata_sim_pre_km[400:580])

clf_svm=svm.SVC(C=1)
clf_svm.fit(train_features_svm,train_labels_svm)
labels_pre_svm=clf_svm.predict(test_features_svm)


score_crossvali=cross_val_score(clf_svm,train_features_svm,train_labels_svm,cv=4)

similarity_metrics=metrics.adjusted_rand_score(labels_pre_svm,test_labels_svm)

print(score_crossvali)

acc_svm=accuracy_score(test_labels_svm,labels_pre_svm)
prn_svm=precision_score(test_labels_svm,labels_pre_svm)
print(acc_svm)
print(prn_svm)





In [18]:
# the classifier clf_svm
def get_depressionlevel_frombio(biodata):
    behavepattern_frombio=clf_svm.predict(biodata)
    return behavepattern_frombio
